In [1]:
import Sloth as sl
import pandas as pd
import numpy as np

In [2]:
pdf = pd.read_csv(r"GOOG.csv", index_col=0, parse_dates=True, infer_datetime_format=True)
sdf = sl.DataFrame.from_pandas(pdf)

In [3]:
%load_ext Cython

In [4]:
pdf

,Open,High,Low,Close,Volume
2004-08-19,100.00,104.06,95.96,100.34,22351900
2004-08-20,101.01,109.08,100.50,108.31,11428600
2004-08-23,110.75,113.48,109.05,109.40,9137200
2004-08-24,111.24,111.60,103.57,104.87,7631300
2004-08-25,104.96,108.00,103.88,106.00,4598900
...,...,...,...,...,...
2013-02-25,802.30,808.41,790.49,790.77,2303900
2013-02-26,795.00,795.95,784.40,790.13,2202500
2013-02-27,794.80,804.75,791.11,799.78,2026100
2013-02-28,801.10,806.99,801.03,801.20,2265800


In [5]:
(pdf.Open > 0).all()

True

In [6]:
pdf.Open.all()

True

In [11]:
(sdf.Open > 1).all()

True

In [ ]:
self.values

In [9]:
s.all()

False

In [ ]:
%%cython
from Sloth.frame cimport Series, DataFrame

cdef Series h_str_1(DataFrame self, str arg):
    return Series(
        # A 1d numpy array
        values=self.values[:, self.columns.get_item(arg)], 
        index=self.index, 
        name=arg
    ) 

def get_1(df, arg):
    return h_str_1(df, arg)

In [ ]:
%%cython
from Sloth.frame cimport Series, DataFrame
cimport numpy as np
import numpy as np

cdef Series h_2(DataFrame self, str arg):
    cdef int col_idx = self.columns.get_item(arg)
    cdef int nrows = self.values.shape[0]
    cdef np.ndarray[np.float64_t, ndim=1] col_values = np.empty(nrows, dtype=np.float64)
    cdef int i

    for i in range(nrows):
        col_values[i] = self.values[i, col_idx]

    return Series(
        values=col_values,
        index=self.index,
        name=arg
    )

def get_2(df, arg):
    return h_2(df, arg)

In [ ]:
%%cython
from Sloth.frame cimport Series, DataFrame
cimport numpy as np
import numpy as np
from libc.stdlib cimport malloc, free


cdef Series h_3(self, arg):
    cdef int col_idx = self.columns.get_item(arg)
    cdef int nrows = self.values.shape[0]
    cdef double* col_values = <double*>malloc(nrows * sizeof(double))
    cdef int i

    # Accessing array elements using pointers
    for i in range(nrows):
        col_values[i] = self.values[i, col_idx]

    # Convert the pointer array to numpy array
    col_values_np = np.asarray(<double[:nrows]>col_values).copy()
    free(col_values)

    return Series(
        values=col_values_np,
        index=self.index,
        name=arg
    )

def get_3(df, arg):
    return h_3(df, arg)

In [ ]:
%timeit get_1(sdf, "Open")

In [ ]:
%timeit get_2(sdf, "Open")

In [ ]:
%timeit get_3(sdf, "Open")

In [ ]:
%%cython 

import numpy as np

int[] 

a = np.array([1, 2, 3]).view()
print(a)

a[0] = 5

print(a)

In [ ]:
%%cython

from libc.stdio cimport printf

cdef int b = 5
cdef int *c = &b
b = 6

print(b, <int> c[0])


In [ ]:
%%cython
import numpy as np
cimport numpy as np

def modify_array(np.ndarray[np.float64_t, ndim=1] arr):
    """
    Create a new NumPy array with modified elements from the input array.
    """
    cdef int n = arr.shape[0]
    cdef np.ndarray[np.float64_t, ndim=1] new_arr = np.empty(n, dtype=np.float64)  # Allocate new array
    cdef double* input_ptr = <double*> arr.data  # Pointer to input array data
    cdef double* new_ptr = <double*> new_arr.data  # Pointer to new array data
    cdef int i
    
    for i in range(n):
        new_ptr[i] = input_ptr[i] * 2  # Example operation: doubling each element
    
    return new_arr


In [ ]:
import numpy as np

# Create a NumPy array
arr = np.array([1.0, 2.0, 3.0, 4.0], dtype=np.float64)

print("Before modification:", arr)

# Call the Cython function
a2 = modify_array(arr)

print("After modification:", a2)


In [ ]:
arr.data

In [ ]:
%timeit modify_array(arr)
%timeit arr * 2